# Adding predefined and custom cross sections to the model and assigning them to beams.

In this example, we show how to 
- add cross sections from the catalog
- add parametric cross sections
- define custom cross sections and add them to the model
- assign a cross section to a beam

## Launch AxisVM and create a new model

We launch an instance of AxisVM.

In [17]:
from axisvm.com.client import start_AxisVM

axisvm_application = start_AxisVM(visible=True, daemon=True)

We import the type library. This is done on a separate line to ensure that the notebook runs correctly for first timers too, since in that case the type library is dynamicall generated with the first launch of an AxisVM instance, which could be the previus block. If this is not the first time you use the library, you can do all the import statements right below each other.

In [18]:
import axisvm.com.tlb as axtlb

Create a new model

In [19]:
axisvm_model_id = axisvm_application.Models.New()
axisvm_model = axisvm_application.Models.Item[axisvm_model_id]

## Add a cross section from the catalog

We can use the cross sections found in the AxisVM cross section library. A cross section can be identified by a parameter describing the shape and the name of the section. For instance, adding an I section to the model looks like this:

In [20]:
section_index_1 = axisvm_model.CrossSections.AddFromCatalog(axtlb.cssI, "I 200")
assert section_index_1 > 0, f"An error occured. Error code: {section_index_1}"

## Add a parametric cross section

Most of the section types can be added with custom values. For instance, adding a parametric asymmetric I section goes like this:

In [21]:
section_index_2 = axisvm_model.CrossSections.AddAsymmetricI(
    "asymmetric I 200",
    0.2,      # cross-section height
    0.09,     # cross-section upper flange width
    0.0075,   # cross-section web thickness
    0.0113,   # cross-section upper flange thickness
    0.04,     # cross-section lower flange width
    0.0113,   # cross-section lower flange thickness
)
assert section_index_2 > 0, f"An error occured. Error code: {section_index_2}"

Adding a parametric box section:

In [22]:
section_index_3 = axisvm_model.CrossSections.AddBox(
    "box 20x10",
    0.2,             # cross-section height
    0.1,             # cross-section width
    0.0075,          # cross-section wall thickness at the web
    0.0113,          # cross-section wall thickness at the flange
    0.0,             # fillet radius
    axtlb.cspWelded  # manufacturing process
)
assert section_index_3 > 0, f"An error occured. Error code: {section_index_3}"

## Create a custom polygonal cross section

We define a rectangular 10cm x 10cm section as a polygon. To define general cross sections, one has to create a list of polygons. If the section is defined by one polygon, the polygon list should be a list of length 1. The polygon list consists of polygons, which consists of lines. Each of these geometrical entities can be created using the `IAxisVMObjectCreator` interface. We use this interface to create interfaces for creating lines, polygons and polygon lists.

In [23]:
# create a polygon list creator object
polygon_list_creator = axisvm_application.ObjectCreator.NewPolygon2dList()

# create a polygon creator object
polygon_creator = axisvm_application.ObjectCreator.NewPolygon2d()

# create a line creator object and add a line to it
# this creates the line between the bottom-left and the bottom-right
# corners of the rectangle
line_creator = axisvm_application.ObjectCreator.NewLine2d()
line_creator.LineType = axtlb.ltStraightLine
line_creator.SetLinePoints(
    axtlb.RPoint2d(
        Coord1 = -0.05,  # in metres
        Coord2 = -0.05,  # in metres
    ),
    axtlb.RPoint2d(
        Coord1 = 0.05,  # in metres
        Coord2 = -0.05,  # in metres
    ),
)
polygon_creator.AddLine(line_creator)

# create a line creator object and add a line to it
# this creates the line between the bottom-right and the top-right
# corners of the rectangle
line_creator = axisvm_application.ObjectCreator.NewLine2d()
line_creator.LineType = axtlb.ltStraightLine
line_creator.SetLinePoints(
    axtlb.RPoint2d(
        Coord1 = 0.05,  # in metres
        Coord2 = -0.05,  # in metres
    ),
    axtlb.RPoint2d(
        Coord1 = 0.05,  # in metres
        Coord2 = 0.05,  # in metres
    ),
)
polygon_creator.AddLine(line_creator)

# create a line creator object and add a line to it
# this creates the line between the top-right and the top-left
# corners of the rectangle
line_creator = axisvm_application.ObjectCreator.NewLine2d()
line_creator.LineType = axtlb.ltStraightLine
line_creator.SetLinePoints(
    axtlb.RPoint2d(
        Coord1 = 0.05,  # in metres
        Coord2 = 0.05,  # in metres
    ),
    axtlb.RPoint2d(
        Coord1 = -0.05,  # in metres
        Coord2 = 0.05,  # in metres
    ),
)
polygon_creator.AddLine(line_creator)

# create a line creator object and add a line to it
# this creates the line between the top-left and the bottom-left
# corners of the rectangle
line_creator = axisvm_application.ObjectCreator.NewLine2d()
line_creator.LineType = axtlb.ltStraightLine
line_creator.SetLinePoints(
    axtlb.RPoint2d(
        Coord1 = -0.05,  # in metres
        Coord2 = 0.05,  # in metres
    ),
    axtlb.RPoint2d(
        Coord1 = -0.05,  # in metres
        Coord2 = -0.05,  # in metres
    ),
)
polygon_creator.AddLine(line_creator)

# add the polygon to the polygon list
polygon_list_creator.Add(polygon_creator)

# add the cross section to the model using the polygon list
custom_section_index_1 = axisvm_model.CrossSections.AddCustomWithUserParams(
    "custom section 1",
    polygon_list_creator,
    axtlb.cspOther,
)[-1]
assert custom_section_index_1 > 0, f"An error occured. Error code: {custom_section_index_1}"

## Create a custom polygonal cross section with a circular hole

We reproduce the previously created cross section, but this time with a circular hole in it. Also, we are going to use some advanced Python features to avoid the repeating steps in the previous section.

In [24]:
# create a polygon list creator object
polygon_list_creator = axisvm_application.ObjectCreator.NewPolygon2dList()

We create a helper function that adds a list of points as a polygon. The function accepts an argument that controls if the polygon represents a hole or not. It is important here, that this attribute must be set after all lines are added to the polygon!

In [25]:
def add_polygon_to_polygon_list(coordinates, hole=False):
    polygon_creator = axisvm_application.ObjectCreator.NewPolygon2d()
    
    for i in range(len(coordinates)-1):
        point_1 = coordinates[i]
        point_2 = coordinates[i+1]
        line_creator = axisvm_application.ObjectCreator.NewLine2d()
        line_creator.LineType = axtlb.ltStraightLine
        line_creator.SetLinePoints(
            axtlb.RPoint2d(Coord1 = point_1[0], Coord2 = point_1[1]),
            axtlb.RPoint2d(Coord1 = point_2[0], Coord2 = point_2[1]),
        )
        polygon_creator.AddLine(line_creator)
    
    # set the polygon as solid or hole
    # this must be set after all lines are added
    polygon_creator.Hole = hole
    
    # add the polygon to the polygon list
    polygon_list_creator.Add(polygon_creator)

We define the same 10cm x 10cm rectangle as before, but now using the helper function we created in the previous block:

In [26]:
# the coordinates of the corner nodes of the rectangle in metres
polygon_coordinates = [
    [-0.05, -0.05],  # bottom-left
    [0.05, -0.05],   # bottom-right
    [0.05, 0.05],    # top-right
    [-0.05, 0.05]    # top-left
]
# add the first point again
polygon_coordinates.append(polygon_coordinates[0])

add_polygon_to_polygon_list(polygon_coordinates)

The hole is a circle with a diameter of 8 cm. Although we could simply use a circular cross section type directly, for the sake of the example we add the circle as a polygon by discretizing the perimeter.

In [27]:
import math

radius_of_circle = 0.04  # in metres
n_segments = 60  # the number of segments

# the coordinates of the perimeter of the circle
polygon_coordinates = []
for i in range(n_segments):
    angle_rad = i*2*math.pi/n_segments
    coord_x = radius_of_circle * math.cos(angle_rad)
    coord_y = radius_of_circle * math.sin(angle_rad)
    polygon_coordinates.append([coord_x, coord_y])
    
# add the first point again
polygon_coordinates.append(polygon_coordinates[0])

add_polygon_to_polygon_list(polygon_coordinates, True)

Add the section to the model:

In [28]:
# add the cross section to the model using the polygon list
custom_section_index_2 = axisvm_model.CrossSections.AddCustomWithUserParams(
    "custom section 2",
    polygon_list_creator,
    axtlb.cspOther,
)[-1]
assert custom_section_index_2 > 0, f"An error occured. Error code: {custom_section_index_1}"

## Define a material

We set the national design code and define a material for later use.

In [29]:
# setting the national design code
axisvm_model.Settings.NationalDesignCode = axtlb.ndcEuroCode

# setting the material
material_index = axisvm_model.Materials.AddFromCatalog(axtlb.ndcEuroCode, "S 235")
assert material_index > 0, f"An error occured. Error code: {material_index}"

## Create the geometry of the beam

We create a line for our beam:

In [30]:
# define the left node
node_index_1 = axisvm_model.Nodes.AddWithDOF(0, 0, 0, axtlb.dofFrameXZ)
assert node_index_1 > 0, f"An error occured. Error code: {node_index_1}"

# define the right node
node_imdex_2 = axisvm_model.Nodes.AddWithDOF(4, 0, 0, axtlb.dofFrameXZ)
assert node_imdex_2 > 0, f"An error occured. Error code: {node_imdex_2}"

# define a line
line_index = axisvm_model.Lines.Add(node_index_1, node_imdex_2, axtlb.lgtStraightLine)[-1]
assert line_index > 0, f"An error occured. Error code: {line_index}"

## Create the beam member

We define our beam using the material and cross section defined before. Assigning a cross section to a beam involves exactly the same steps, regardless of the way it was defined. You can use any of the previously defined cross sections by referring to it using the appropriate index.

In [31]:
# add the member defined by a single line
member_index = axisvm_model.Members.Add([line_index])
assert member_index > 0, f"An error occured. Error code: {member_index}"

# access the newly created member
member = axisvm_model.Members.Item[member_index]

# define the beam
beam_index = member.DefineAsBeam(
    material_index,          # the index of the material
    custom_section_index_2,  # start cross section index
    custom_section_index_2,  # end cross section index  
)[-1]
assert beam_index > 0, f"An error occured. Error code: {beam_index}"

At the end of our workflow, we close AxisVM.

In [16]:
axisvm_application.Quit()